In [1]:
import pandas as pd
import os
import yaml
import pyreadr
import numpy as np
import pdb

# Pickup all excel sheets

In [2]:
with open('./config_echoes.yaml', 'r') as f:
    args = yaml.safe_load(f)

data_files = []
for _dir in os.listdir(args['data_dir']):
    for filename in os.listdir(os.path.join(args['data_dir'], _dir)):
        if args['format'] == 'rda': # scan for .rdata/.rda
            if filename.endswith(".rda") or filename.endswith(".rdata"): 
                data_files.append(os.path.join(args['data_dir'], _dir, filename))
            else:
                continue
        elif args['format'] == 'xls': # scan for .xls/.xlsx
            if filename.endswith(".xls") or filename.endswith(".xlsx"): 
                 data_files.append(os.path.join(args['data_dir'], _dir, filename))
            else:
                continue
        else:
            print('Unsupported file format selcted')

print(data_files)
print(len(data_files))

['/media/windowsC/SIN_Desktop/ENCHANT/ECHOES/data/Energy company survey/ECHOES_raw_data_survey_energy_provider.xls', '/media/windowsC/SIN_Desktop/ENCHANT/ECHOES/data/International survey/ECHOES_Main_survey_codebook.xlsx', '/media/windowsC/SIN_Desktop/ENCHANT/ECHOES/data/International survey/ECHOES_raw_data_int_survey.xlsx', '/media/windowsC/SIN_Desktop/ENCHANT/ECHOES/data/Psychological experiment Spain/ECHOES_raw_data_psych_experiment_Spain.xlsx', '/media/windowsC/SIN_Desktop/ENCHANT/ECHOES/data/Standardized psychological experiments/ECHOES_raw_data_psych_experiments_Germany 1.xlsx', '/media/windowsC/SIN_Desktop/ENCHANT/ECHOES/data/Standardized psychological experiments/ECHOES_raw_data_psych_experiments_Bulgaria.xlsx', '/media/windowsC/SIN_Desktop/ENCHANT/ECHOES/data/Standardized psychological experiments/ECHOES_raw_data_psych_experiments_Finland.xlsx', '/media/windowsC/SIN_Desktop/ENCHANT/ECHOES/data/Standardized psychological experiments/ECHOES_raw_data_psych_experiments_Germany 2.xl

# Sanity checks and iterate over individual sheet

In [3]:
import subprocess

def get_survey_df_list(_data_files, 
                       remove_files_with_phrases=['codebook', 'codes_labels']):
    survey_df_dict = {}
    data_files = [_file for _file in _data_files if not any(phrase in _file for phrase in remove_files_with_phrases)]
        
    for _file in data_files:
        try:
            sheet = pd.read_excel(_file, sheet_name=0) # Access the first sheet the other are codes and questionaire interpretations
        except Exception as e:
            print('Exception caught in xls')

            _csv = '.'.join(_file.split('.')[:-1])+'.csv'
            if os.path.exists(_csv+'.0'):
                print('File exists: Skip xls to csv conversion')
            else:
                #pdb.set_trace()
                # Try converting xls to csv through CLI
                cmd = ['ssconvert', '-S', _file, _csv]
                proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

                o, e = proc.communicate()

            sheet = pd.read_csv(_csv+'.0')
        
        name = _file.replace(" ", "_").split('/')[-1]
        survey_df_dict[name] = sheet
        sheet = None

    return survey_df_dict

survey_df_dict = get_survey_df_list(data_files)

Exception caught in xls
File exists: Skip xls to csv conversion


/home/kalyan/anaconda3/envs/deepl/lib/python3.7/site-packages/ipykernel_launcher.py:33: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.


Exception caught in xls
File exists: Skip xls to csv conversion
Exception caught in xls
File exists: Skip xls to csv conversion
Exception caught in xls
File exists: Skip xls to csv conversion
Exception caught in xls
File exists: Skip xls to csv conversion
Exception caught in xls
File exists: Skip xls to csv conversion
Exception caught in xls
File exists: Skip xls to csv conversion
Exception caught in xls
File exists: Skip xls to csv conversion


In [4]:
for name, survey_df in survey_df_dict.items():
    print(f"{name}: {survey_df.shape}")

ECHOES_raw_data_survey_energy_provider.xls: (181, 94)
ECHOES_raw_data_int_survey.xlsx: (18037, 424)
ECHOES_raw_data_psych_experiment_Spain.xlsx: (302, 52)
ECHOES_raw_data_psych_experiments_Germany_1.xlsx: (200, 111)
ECHOES_raw_data_psych_experiments_Bulgaria.xlsx: (157, 91)
ECHOES_raw_data_psych_experiments_Finland.xlsx: (196, 76)
ECHOES_raw_data_psych_experiments_Germany_2.xlsx: (208, 132)
ECHOES_raw_data_psych_experiments_Italy.xlsx: (178, 169)
ECHOES_raw_data_psych_experiments_Norway.xlsx: (250, 74)
ECHOES_raw_data_psych_experiments_Turkey.xlsx: (211, 107)


In [5]:
import inspect

class ConditionalSurvey(object):
    
    def get_nan_idices(self, sheet):
        '''
        '''
        idx = np.argwhere(pd.isnull(sheet).to_numpy()) #2D indices
        #print(idx)
        return idx
    
    
    def no_nan(self, _sheet):
        '''
        '''
        sheet = _sheet.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
        #print(sheet.shape)
        return sheet

    def substitute_nan(self, _sheet):
        '''
        '''
        sheet = _sheet.fillna(0.0)
        #print(sheet.shape)
        return sheet
    
    
def get_methods(class_arg):

    assert inspect.isclass(class_arg), 'Expecting a class as an argument'
    
    method_list = []
    # attribute is a string representing the attribute name
    for attribute in dir(class_arg):
        # Get the attribute value
        attribute_value = getattr(class_arg, attribute)
        # Check that it is callable
        if callable(attribute_value):
            # Filter all dunder (__ prefix) methods
            if attribute.startswith('__') == False:
                method_list.append(attribute)

    return method_list

In [6]:
inst = ConditionalSurvey()
methods_list = get_methods(ConditionalSurvey)
print(methods_list)

['get_nan_idices', 'no_nan', 'substitute_nan']


In [7]:
print(args)

{'format': 'xls', 'data_dir': '/media/windowsC/SIN_Desktop/ENCHANT/ECHOES/data', 'surveys': {'survey_energy_provider': {'histogram': {'enable': True, 'ROWS': 10, 'COLS': 10, 'SIZE': ['8*8', '6*8']}, 'conditional': 'substitute_nan'}, 'int_survey': {'histogram': {'enable': True, 'ROWS': 23, 'COLS': 19, 'SIZE': ['8*10', '6*10']}, 'conditional': 'substitute_nan'}, 'psych_experiment_Spain': {'histogram': {'enable': True, 'ROWS': 5, 'COLS': 11, 'SIZE': ['8*8', '6*8']}, 'conditional': 'substitute_nan'}, 'psych_experiments_Germany_1': {'histogram': {'enable': True, 'ROWS': 11, 'COLS': 11, 'SIZE': ['8*8', '6*8']}, 'conditional': 'substitute_nan'}, 'psych_experiments_Germany_2': {'histogram': {'enable': True, 'ROWS': 11, 'COLS': 12, 'SIZE': ['8*8', '6*8']}, 'conditional': 'substitute_nan'}, 'psych_experiments_Bulgaria': {'histogram': {'enable': True, 'ROWS': 10, 'COLS': 10, 'SIZE': ['8*8', '6*8']}, 'conditional': 'substitute_nan'}, 'psych_experiments_Finland': {'histogram': {'enable': True, 'ROW

In [10]:
import matplotlib.pyplot as plt
def build_histogram(parsed_df,
                    settings,
                   survey_name,
                   ignore_label=0):
    #df = pd.DataFrame({'species': ['bear', 'bear', 'marsupial'], 'population': [1864, 22000, 80000]}, index=['panda', 'polar', 'koala'])
    parsed_df = parsed_df.iloc[settings.get("n_discard_columns", 1):, :]
    ignore_labels = settings.get("ignore_labels", [ignore_label])
    
    ROWS, COLS, SIZE = settings.get("ROWS", 0), settings.get("COLS", 0), settings.get("SIZE", (['8*8', '6*8']))
    assert not (ROWS == 0 and COLS == 0), f'{survey_name} Error: ROWS/COL value not specified in config.yaml'
    
    SIZE = list(map(lambda x: eval(x), SIZE))
    fig, axes = plt.subplots(figsize=SIZE, dpi=300, nrows=ROWS, ncols=COLS)
    idx = 0
    for name, col in parsed_df.items():
        #assert all(col.map(type) == int) or all(col.map(type) == float)
        sr_count = col.value_counts()
        _ignore_labels = [label for label in ignore_labels if label in sr_count.keys()]
        if len(_ignore_labels) > 0:
            sr_count.drop(labels=_ignore_labels, inplace=True)
            #pdb.set_trace()
        
        if sr_count.sum() > 0:  #Display opitions with responses
            i, j = idx // COLS, idx % COLS
            idx += 1
            if i>= ROWS or j>= COLS:
                pdb.set_trace()

            #print(sr_count.keys().dtype, sr_count.values.dtype)
            ax = axes[i, j]
            FIRST_TIME = True
            while FIRST_TIME:
                try:
                    ax.bar(sr_count.keys(), sr_count.values, align='center', color='#607c8e')
                    ax.plot(sr_count.keys(), sr_count.values, marker="o", linestyle="", alpha=0.8, color="b")
                    FIRST_TIME = False
                except TypeError:
                    if FIRST_TIME:
                        sr_count.index = list(map(str, list(sr_count.keys())))
                        FIRST_TIME = False
                    else:
                        pdb.set_trace()
            ax.set_title(f'{name}', fontsize=7, fontweight='bold', color='blue')

            ax.xaxis.set_tick_params(labelsize=5, labelcolor='r')
            ax.set_xlabel(f'response[{sr_count.sum()}]', fontsize=5, fontweight='bold')
            ax.yaxis.set_tick_params(labelsize=5, labelcolor='r')
            ax.set_ylabel('count', fontsize=5, fontweight='bold')
            ax.grid(axis='y', alpha=0.75)
    
    fig.tight_layout(pad=3.0)
    plt.savefig(f'{survey_name}.png', bbox_inches='tight')
    plt.close()
    
for key, val in args['surveys'].items():
    assert val['conditional'] in methods_list, 'Error: Improper conditional parsing method in config.yaml'
    conditional_parser = getattr(ConditionalSurvey, val['conditional'])
    print(f"{key}: {conditional_parser}")
    
    matching_df = [(file_name, survey_df) for file_name, survey_df in survey_df_dict.items() if key in file_name]
    assert len(matching_df) > 0, 'Error: Improper survey name in config.yaml'
    parsed_df = conditional_parser(inst, matching_df[0][1])
    print(f"before:{matching_df[0][1].shape}, after:{parsed_df.shape}")
    
    # Build a histogram of each column in df
    if val['histogram']['enable']:
        build_histogram(parsed_df, val['histogram'], f"{key}_{val['conditional']}")

survey_energy_provider: <function ConditionalSurvey.substitute_nan at 0x7fcda4e46560>
before:(181, 94), after:(181, 94)
int_survey: <function ConditionalSurvey.substitute_nan at 0x7fcda4e46560>
before:(18037, 424), after:(18037, 424)
psych_experiment_Spain: <function ConditionalSurvey.substitute_nan at 0x7fcda4e46560>
before:(302, 52), after:(302, 52)
psych_experiments_Germany_1: <function ConditionalSurvey.substitute_nan at 0x7fcda4e46560>
before:(200, 111), after:(200, 111)
psych_experiments_Germany_2: <function ConditionalSurvey.substitute_nan at 0x7fcda4e46560>
before:(208, 132), after:(208, 132)


/home/kalyan/anaconda3/envs/deepl/lib/python3.7/site-packages/ipykernel_launcher.py:52: UserWarning: Tight layout not applied. tight_layout cannot make axes width small enough to accommodate all axes decorations


psych_experiments_Bulgaria: <function ConditionalSurvey.substitute_nan at 0x7fcda4e46560>
before:(157, 91), after:(157, 91)
psych_experiments_Finland: <function ConditionalSurvey.substitute_nan at 0x7fcda4e46560>
before:(196, 76), after:(196, 76)
psych_experiments_Italy: <function ConditionalSurvey.substitute_nan at 0x7fcda4e46560>
before:(178, 169), after:(178, 169)
psych_experiments_Norway: <function ConditionalSurvey.substitute_nan at 0x7fcda4e46560>
before:(250, 74), after:(250, 74)
psych_experiments_Turkey: <function ConditionalSurvey.substitute_nan at 0x7fcda4e46560>
before:(211, 107), after:(211, 107)
